In [42]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [3]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

In [7]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [12]:
x_train

,age,gender,fever,cough,city
2,42,Male,101.0,Mild,Delhi
66,51,Male,104.0,Mild,Kolkata
90,59,Female,99.0,Strong,Delhi
35,82,Female,102.0,Strong,Bangalore
41,82,Male,NaN,Mild,Kolkata
...,...,...,...,...,...
25,23,Male,NaN,Mild,Mumbai
27,33,Female,102.0,Strong,Delhi
21,73,Male,98.0,Mild,Bangalore
72,83,Female,101.0,Mild,Kolkata


<h2>Using Transformers without Column Transformer Class</h2>

In [14]:
#adding simple imputer to fever col

In [15]:
si=SimpleImputer()
si

SimpleImputer()

In [24]:
x_train_fever=si.fit_transform(x_train[['fever']])
x_train_fever[0:5]

array([[101.        ],
       [104.        ],
       [ 99.        ],
       [102.        ],
       [100.91666667]])

In [25]:
x_train_fever.shape

(80, 1)

In [26]:
x_test_fever=si.fit_transform(x_test[['fever']])
x_test_fever[0:5]

array([[103.],
       [ 98.],
       [100.],
       [ 99.],
       [ 99.]])

In [27]:
x_test_fever.shape

(20, 1)

In [28]:
#OrdinalEncoding -> Cough col

In [31]:
oe=OrdinalEncoder(categories=[['Mild','Strong']])
oe

OrdinalEncoder(categories=[['Mild', 'Strong']])

In [34]:
x_train_cough=oe.fit_transform(x_train[['cough']])
x_train_cough[0:5]

array([[0.],
       [0.],
       [1.],
       [1.],
       [0.]])

In [35]:
x_train_cough.shape

(80, 1)

In [36]:
x_test_cough=oe.fit_transform(x_test[['cough']])
x_test_cough[0:5]

array([[0.],
       [1.],
       [0.],
       [0.],
       [1.]])

In [37]:
x_test_cough.shape

(20, 1)

In [38]:
#OneHotEncoding -> gender,city

In [39]:
ohe=OneHotEncoder(drop='first',sparse=False)
ohe

OneHotEncoder(drop='first', sparse=False)

In [43]:
x_train_gender_city = ohe.fit_transform(x_train[['gender','city']])

# also the test data
x_test_gender_city = ohe.fit_transform(x_test[['gender','city']])

x_train_gender_city.shape

(80, 4)

In [47]:

# Extracting Age
x_train_age = x_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
x_test_age = x_test.drop(columns=['gender','fever','cough','city']).values

x_train_age.shape

(80, 1)

In [48]:
x_train_transformed = np.concatenate((x_train_age,x_train_fever,x_train_gender_city,x_train_cough),axis=1)
# also the test data
x_test_transformed = np.concatenate((x_test_age,x_test_fever,x_test_gender_city,x_test_cough),axis=1)

x_train_transformed.shape

(80, 7)

<h2>USING COLUMN TRANSFORMER</h2>

In [49]:
from sklearn.compose import ColumnTransformer

In [50]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [52]:
transformer.fit_transform(x_train).shape

(80, 7)

In [53]:
transformer.transform(x_test).shape

(20, 7)